# Scenario models

For a delta-change in a feature (or group of features), what is the likely change in displacement?

### Assumptions:
- changes and impacts occur in year=0, year=1. 
- User-specified change can be 
  - A centered quantile value
  - A percentage change, 
  - or even an absolute value.
  

### Set up
Consider a limited (simplified) model to estimate elasticities

In [28]:
import pandas as pd
from time import time
import os
import json
import pickle
import numpy as np 
from time import time
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

# Data transforms

<TBC>

In [29]:
start_time = time()
with open("../configuration.json", 'rt') as infile:
    config = json.load(infile)


sources = [os.path.join("..", config['paths']['output'], 
                        d['name'], 
                        'data.csv') for d in config['sources'] if d['name']]


# Generate a data frame with all indicators
df = pd.concat((pd.read_csv(f) for f in sources), sort=False, ignore_index=True)

# Summary stats
print("Sources            : {}".format(len(sources)))
print("Shape              : {} (rows) {} (columns)".format(*df.shape))
print("Geographies        : {}".format(len(df['Country Name'].unique())))
print("Indicators         : {}".format(len(df['Indicator Code'].unique())))
print("Temporal coverage  : {} -> {}".format(df.year.min(), df.year.max()))
print("Null values        : {}".format(sum(df['value'].isnull())))

print("\nLoaded data in {:3.2f} sec.".format(time() - start_time))


# Now arrange data in long form
data = pd.pivot_table(df, index=['Country Code', 'year'],
                      columns='Indicator Code', values='value')

# Consider country/year as features (and not an index)
data.reset_index(inplace=True)

print("Long form of size  : {} (rows) {} (columns)".format(*data.shape))

Sources            : 19
Shape              : 1016586 (rows) 6 (columns)
Geographies        : 562
Indicators         : 151
Temporal coverage  : 1789 -> 2019
Null values        : 1421

Loaded data in 1.31 sec.
Long form of size  : 35856 (rows) 153 (columns)


In [30]:
# Only look at features used in scenarios
groupings = json.load(open("../groupings.json", 'rt'))
featureset = [i['code'] for c in groupings['clusters'] for i in c['indicators']]

In [31]:
# Define the subset to work with

countries = ['AFG', 'MMR']

# Features
idx = ['Country Code', 'year']
mm = ['ETH.TO.{}'.format(i) for i in ['DNK', 'GBR', 'ITA', 'SAU', 'SWE', 'ZAF']]
endo = ['UNHCR.OUT.AS', 'UNHCR.OUT.IDP', 'UNHCR.OUT.OOC', 
        'UNHCR.OUT.REF', 'UNHCR.OUT.RET', 'UNHCR.OUT.RETIDP']
# missing entirely
emdat = ['EMDAT.CPX.OCCURRENCE','EMDAT.CPX.TOTAL.DEATHS','EMDAT.CPX.TOTAL.AFFECTED','EMDAT.CPX.AFFECTED']
target = ['DRC.TOT.DISP']
allfeatures = list(set(featureset) - set(idx + mm + endo + target + emdat))

mmr_data = [f for f in allfeatures if f.startswith('MMR.NSO')]

features = {'AFG': list(set(allfeatures) - set(mmr_data)), 'MMR': allfeatures}

# filter
c1 = data['Country Code'].isin(countries)
c2 = data.year >= 1980

df = data.loc[c1 & c2, idx + allfeatures + target]
print("Filtered data has {} rows, {} columns.".format(*df.shape))

Filtered data has 80 rows, 16 columns.


# Features

In [59]:
def lag_variables(data, var, lag):
    """
    Append lagged variables to frame.
    
    data - pandas data frame
    var - list of variable names to lag
    lag - integer
    
    """
    idx_cols = ['year', 'Country Code']
    fv = var + idx_cols
    
    tmp = data[fv].copy(deep=True)
    
    col_name = [v + ".T" + "{0:+}".format(lag) for v in var]
    
    tmp.rename(columns={k: v for (k, v) in zip(var, col_name)},
               inplace=True)
    tmp.year -= lag
    data = pd.merge(data, tmp, on=idx_cols, how='left')
    
    return data, col_name

# Models

In [61]:
df = df2.copy(deep=True)

df, varname = lag_variables(df, ['DRC.TOT.DISP'], 1)

In [65]:
varname

['DRC.TOT.DISP.T+1']

In [62]:
TARGETS = ['DRC.TOT.DISP']

df2 = df.copy(deep=True)

srcdata = {}
for c in countries:
    
    c1 = df['Country Code'] == c
    
    X = df.loc[c1, featureset + []]
    Y = df.loc[c1, TARGETS]
    print(pd.isnull(X).sum())
    
    # handle missing labels
    idx = ~pd.isnull(Y.values)
    Y = Y[idx]
    X = X[idx]
    print(sum(idx))
    
    # Missing data
    X.fillna(X.mean(), inplace=True)
    Y.fillna(Y.mean(), inplace=True)
    srcdata[c] = X, Y
    
    print("Country: {}, training data: {} rows {} cols".format(c, *X.shape))
    

Indicator Code
NY.GDP.PCAP.PP.CD       23
FP.CPI.TOTL.ZG          27
SL.UEM.TOTL.ZS          11
VC.BTL.DETH             10
VDEM.FRD.POL.KILL        1
DRC.CORR.INDEX          24
HR.SCR.MEAN              9
FSI.STA.LEG             26
FSI.PUB.SER             26
EMDAT.NAT.OCCURRENCE     1
ER.H2O.INTR.PC          32
SP.POP.GROW              1
SP.URB.GROW              1
dtype: int64
[39]
Country: AFG, training data: 39 rows 13 cols
Indicator Code
NY.GDP.PCAP.PP.CD       11
FP.CPI.TOTL.ZG           7
SL.UEM.TOTL.ZS          11
VC.BTL.DETH             10
VDEM.FRD.POL.KILL        1
DRC.CORR.INDEX          24
HR.SCR.MEAN              9
FSI.STA.LEG             26
FSI.PUB.SER             26
EMDAT.NAT.OCCURRENCE     1
ER.H2O.INTR.PC          32
SP.POP.GROW              1
SP.URB.GROW              1
dtype: int64
[39]
Country: MMR, training data: 39 rows 13 cols


In [63]:
for c in countries:
    c1 = df['Country Code'] == c
    k = pd.isnull(df[c1]).sum()
    #print(k)
    X, Y = srcdata[c]
    print(pd.isnull(X).sum())

Indicator Code
NY.GDP.PCAP.PP.CD       0
FP.CPI.TOTL.ZG          0
SL.UEM.TOTL.ZS          0
VC.BTL.DETH             0
VDEM.FRD.POL.KILL       0
DRC.CORR.INDEX          0
HR.SCR.MEAN             0
FSI.STA.LEG             0
FSI.PUB.SER             0
EMDAT.NAT.OCCURRENCE    0
ER.H2O.INTR.PC          0
SP.POP.GROW             0
SP.URB.GROW             0
dtype: int64
Indicator Code
NY.GDP.PCAP.PP.CD       0
FP.CPI.TOTL.ZG          0
SL.UEM.TOTL.ZS          0
VC.BTL.DETH             0
VDEM.FRD.POL.KILL       0
DRC.CORR.INDEX          0
HR.SCR.MEAN             0
FSI.STA.LEG             0
FSI.PUB.SER             0
EMDAT.NAT.OCCURRENCE    0
ER.H2O.INTR.PC          0
SP.POP.GROW             0
SP.URB.GROW             0
dtype: int64


In [64]:
models = {}
for c in countries:
    X, Y = srcdata[c]
    X1 = sm.add_constant(X)
    models[c] = sm.OLS(Y, X1)
    result = models[c].fit()
    print(result.summary())
    

                            OLS Regression Results                            
Dep. Variable:           DRC.TOT.DISP   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     2.649
Date:                Wed, 19 Feb 2020   Prob (F-statistic):             0.0177
Time:                        23:04:52   Log-Likelihood:                -597.00
No. Observations:                  39   AIC:                             1222.
Df Residuals:                      25   BIC:                             1245.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 6.842e+06 

/Users/rahul/anaconda3/envs/mm4sight/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [57]:
result.params

const                   2.467383e+06
NY.GDP.PCAP.PP.CD      -6.543162e+01
FP.CPI.TOTL.ZG          1.805991e+03
SL.UEM.TOTL.ZS          7.343575e+05
VC.BTL.DETH             4.938279e+01
VDEM.FRD.POL.KILL       2.517620e+05
DRC.CORR.INDEX          1.939123e+04
HR.SCR.MEAN             1.544336e+05
FSI.STA.LEG            -9.046677e+04
FSI.PUB.SER             4.706371e+04
EMDAT.NAT.OCCURRENCE   -5.249913e+04
ER.H2O.INTR.PC         -3.952726e+00
SP.POP.GROW            -5.384398e+05
SP.URB.GROW            -5.248928e+05
dtype: float64